# A simple nerual network for fun!

### Definations:

$N_n$: The number of nodes in $n$-$th$ layer.

$X$: The input of ANN, equals to $O^{(1)}$. Shape: ($M$, $N_1$)

$O^{(n)}$: The output of $n$-$th$ layer, also the input of $(n+1)$-$th$ layer. Shape: $(M,N_n)$

$W^{(n)}$: The weight of $n$-$th$ layer. Shape: $(N_n, N_{n+1})$

$B^{(n)}$: The bias of $n$-$th$ layer. Shape: $(N_{n+1},)$

$A^{(n)}$: The intermediate value in each layer, equals to $O^{(n)}W^{(n)}+B^{(n)}$. Shape: $(M, N_{n+1})$

### Euqaitons:
 * Forward Propagation
\begin{align}
    O^{(n+1)} & = Active(A^{(n)}) \\
              & = Active(O^{(n)}W^{(n)}+B^{(n)}) \\
\end{align}

 * Backward Propagation
\begin{align}
    \frac{\partial E}{\partial A^{(n)}} &= \frac{\partial E}{\partial O^{(n+1)}} 
    \odot Active^\prime(A^{(n)}) \\
    \\
    \frac{\partial E}{\partial O^{(n)}} &= \frac{\partial E}{\partial A^{(n)}} {W^{(n)}}^T \\
    \\
    \frac{\partial E}{\partial W^{(n)}} &= {O^{(n)}}^T\frac{\partial E}{\partial A^{(n)}} \\
    \\
    \frac{\partial E}{\partial B^{(n)}} &= \sum_{m}\big( \frac{\partial E}{\partial A^{(n)}} \big)_{m}
\end{align}

*Matrix derivative uses denominator alignment.

In [9]:
import numpy as np

delta = 1e-15

In [10]:
# Util functions
def label2bin(y):
    shape = (y.shape[0], np.max(y) + 1)
    rst = np.zeros(shape, dtype = int)
    for idx, label in enumerate(y):
        rst[idx, label] = 1
    return rst

def maxCol(X):
    return np.max(X, axis = 1)[:, np.newaxis]

def sumCol(X):
    return np.sum(X, axis = 1)[:, np.newaxis]

def averCol(X):
    return np.average(X, axis = 1)[:, np.newaxis]

In [11]:
"""
Pairs of math functions.
f(x) returns the values for forward propagation. 
Df(x, dr) applies chain rule and returns derivatives of parameters for backward propagation.
"""
def mul(X, Y):
    return X.dot(Y)

def Dmul(X, Y, dLdO):
    return dLdO.dot(Y.T), X.T.dot(dLdO)

def relu(A):
    return A * (A > 0)

def Drelu(A, dLdO, O = None):
    return dLdO * (A > 0)

def sigmoid(A):
    y = 1 / (1 + np.exp(-A))
    return y

def Dsigmoid(A, dLdO, O = None):
    if O is None:
        O = sigmoid(A)
    return dLdO * O * (1 - O)

def softMax(A):
    shift = A - maxCol(A)
    exp = np.exp(shift)
    return exp / sumCol(exp)

def DsoftMax(A, dLdO, O = None):
    if O is None:
        O = softMax(A)
    O_dLdO = O * dLdO
    return O_dLdO - O * sumCol(O_dLdO)

activFunc = {
    'relu': relu,
    'sigmoid': sigmoid,
    'softMax': softMax,
}

DactivFunc = {
    'relu': Drelu,
    'sigmoid': Dsigmoid,
    'softMax': DsoftMax,
}

In [12]:
"""
Loss functions.
    inputs(
        O: Output layer
        y: True value
    )
    return(
        loss: Loss of the neural network.(without regularization)
        dLdO: Derivative of loss with respect of O
        dLdA: Derivative of loss with respect of A
    )
"""
def LSM(O, y):
    assert(O.shape == y.shape)
    M = O.shape[0]
    dLdO = O - y
    loss = np.sum(dLdO**2) / (2 * M)
    return loss, dLdO, None

# y is class label.
def labelCrossEntropy(O, y):
    M = O.shape[0]
    dLdO = -label2bin(y) / (O + delta)
    loss = -np.sum(np.log(np.choose(y, O.T) + delta)) / M
    return loss, dLdO, None

# y is class probability.
def probCrossEntropy(O, y):
    assert(O.shape == y.shape)
    M = O.shape[0]
    dLdO = -y / (O + delta)
    loss = -np.sum(np.log(O + delta) * y) / M
    return loss, dLdO, None

# Output layer is softMax
def softMaxLabelCrossEntropy(O, y):
    M = O.shape[0]
    loss = -np.sum(np.log(np.choose(y, O.T) + delta)) / M
    return loss, None, (O - label2bin(y))

def softMaxProbCrossEntropy(O, y):
    assert(O.shape == y.shape)
    loss = -np.sum(np.log(O + delta) * y) / M
    return loss, None, (O - y)

lossFunc = {
    'LSM': LSM,
    'labelCrossEntropy': labelCrossEntropy,
    'probCrossEntropy': probCrossEntropy,
    'softMaxLabelCrossEntropy': softMaxLabelCrossEntropy,
    'softMaxProbCrossEntropy': softMaxProbCrossEntropy,
}

In [13]:
"""
    A layer of nerual network
"""
class layer:
    def __init__(self, shape, activ, isOutLayer = False, loss = None, reg = 1e-1, std = 1e-1):
        assert(len(shape) == 2)
        self.W = std * np.random.randn(*shape)
        self.B = np.zeros(shape[1])
        self.reg = reg
        self.activ = activFunc[activ]
        self.Dactiv = DactivFunc[activ]
        self.isOutLayer = isOutLayer
        if isOutLayer:
            self._loss = lossFunc[loss]
        
    def forward(self, X):
        self.X = X
        self.A = mul(X, self.W) + self.B
        self.O = self.activ(self.A)
        return self.O
    
    def backward(self, rate, dLdO = None):
        dLdA = None
        if self.isOutLayer:
            dLdA = self.dLdA
            dLdO = self.dLdO
        
        if dLdA is None:
            dLdA = self.Dactiv(self.A, dLdO, self.O)
        
        M = self.O.shape[0]
        dLdB = np.average(dLdA, axis = 0)
        dLdO, dLdW = Dmul(self.X, self.W, dLdA)
        
        self.B -= rate * dLdB
        self.W -= rate * (dLdW / M + self.reg * self.W)

        return dLdO
    
    def regLoss(self):
        return self.reg * np.sum(self.W**2)/2
    
    def loss(self, y):
        rst, self.dLdO, self.dLdA =  self._loss(self.O, y)
        return rst
"""
    neural network class
"""
class simpleNN:
    """
        init a neural network.
        inputs(
            layers:  A list of dict, i.e. 
                        [{   
                             shape : (10, 20)
                             activ : 'sigmoid',
                         },
                         {   
                             shape : (20, 40)
                             activ : 'sigmoid',
                         },
                         {   
                             shape : (40, 10)
                             activ : 'sigmoid',
                             loss  : 'LSM',
                             isOutLayer : True
                             
                         }]
                    It specifies the node number and activation function of each layer. 
                    For output layer, the loss function should also be given.
        )
    """
    def __init__(self, layers):
        ly = []
        for l in layers:
            ly.append(layer(l['shape'], l['activ'], l.get('isOutLayer', False), l.get('loss', None)))
        self.layers = ly
    
    def forward(self, X, y = None):
        loss = 0
        for l in self.layers:
            X = l.forward(X)
            #loss += l.regLoss()
        
        if y is not None:
            loss += self.layers[-1].loss(y)
        
        return X, loss

    def backward(self, step):
        dLdO = None
        for l in reversed(self.layers):
            dLdO = l.backward(step, dLdO)
    
    def train(self, X_train, y_train, X_test, y_test, rate = 1e-3, epochs = 20, batchSz = 200, rateDecay = 1):
        num = X_train.shape[0]
        ite = int(max(1, num / batchSz))
        for epoch in range(epochs):
            perm = np.random.permutation(num)
            for i in range(ite):
                ids = perm[i * batchSz: (i + 1) * batchSz]
                X_batch = X_train[ids, :]
                y_batch = y_train[ids]
                
                pred, loss = self.forward(X_batch, y_batch)
                self.backward(rate)
                
            rate *= rateDecay
            
            print('------------------epoch:', epoch, '------------------' )
            
            pred, loss = self.forward(X_train, y_train)
            print('train_accuracy: %.5f' % np.mean(np.argmax(pred, axis = 1) == y_train), ' loss: %.5f'% loss)
            
            pred, loss = self.forward(X_test, y_test)
            print('test_accuracy : %.5f' % np.mean(np.argmax(pred, axis = 1) == y_test), ' loss: %.5f\n'% loss)

In [14]:
layers = [
    {
        'shape': (784, 100),
        'activ': 'relu',
    },
    {
        'shape': (100, 100),
        'activ': 'relu',
    },
    {
        'shape': (100, 10),
        'activ': 'softMax',
        'isOutLayer' : True,
        'loss' : 'softMaxLabelCrossEntropy'
    }
]
snn = simpleNN(layers)

In [15]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST-original', data_home='datasets/mnist')
X = mnist['data'].astype(float)
y = mnist['target'].astype(int)
I = np.random.permutation(X.shape[0])
X = X[I, :]
y = y[I]

X_train = X[:50000, :]
y_train = y[:50000]
X_test = X[50000:, :]
y_test = y[50000:]

In [16]:
snn.train(X_train, y_train, X_test, y_test, epochs = 30)

------------------epoch: 0 ------------------
train_accuracy: 0.80826  loss: 1.58909
test_accuracy : 0.79310  loss: 1.79863

------------------epoch: 1 ------------------
train_accuracy: 0.82760  loss: 0.84931
test_accuracy : 0.81525  loss: 0.98916

------------------epoch: 2 ------------------
train_accuracy: 0.84424  loss: 0.62283
test_accuracy : 0.82930  loss: 0.73447

------------------epoch: 3 ------------------
train_accuracy: 0.86206  loss: 0.50067
test_accuracy : 0.84895  loss: 0.59153

------------------epoch: 4 ------------------
train_accuracy: 0.87808  loss: 0.43555
test_accuracy : 0.86190  loss: 0.52215

------------------epoch: 5 ------------------
train_accuracy: 0.88976  loss: 0.38440
test_accuracy : 0.87440  loss: 0.46551

------------------epoch: 6 ------------------
train_accuracy: 0.89312  loss: 0.35875
test_accuracy : 0.87870  loss: 0.43693

------------------epoch: 7 ------------------
train_accuracy: 0.90048  loss: 0.33104
test_accuracy : 0.88690  loss: 0.40607

